n : 지점개수
s : 합승시작점
a : a목적지
b : b목적지

In [10]:
from collections import defaultdict
from heapq import *
def mkgraph(fares):
    graph = defaultdict(list)
    for fr,to,wt in fares:
        graph[fr].append((wt,fr,to))
        graph[to].append((wt,to,fr))
    return graph

def dijkstra(graph,start):
    distances = {vertex : float('inf') for vertex in graph}
    distances[start] = 0
    frm = {i:'X' for i in graph}
    frm[start] = start

    hpq = [[0,start]]

    while hpq : 
        curDist, curVertex = heappop(hpq)
        if distances[curVertex] < curDist : continue
        if graph.get(curVertex):
            for wt, fr, to in graph.get(curVertex):
                new_dist = curDist + wt
                if new_dist < distances[to]:
                    distances[to] = new_dist
                    frm[to] = curVertex
                    hpq.append([new_dist, to])

    return distances, frm
        
def find_path(distances,frm, a, b):# a to b path
    path=[]
    ans=[]
    now = b
    while True:
        # print(now)
        path.append(now)
        now = frm[now]
        if now==a:
            path.append(now)
            break
    while path:
        ans.append(path.pop())
    # print(ans, distances[b])
    return ans, distances[b]


def solution(n,s,a,b,fares):
    answer = []
    graph = mkgraph(fares)
    
    # dist_fr_start , path_fr_start= dijkstra(graph, s)
    
    # dist_fr_a, path_fr_a = dijkstra(graph, a)
    # path_a2b , withpay = find_path(dist_fr_a, path_fr_a, a , b)
    
    
    # tmp = min(dist_fr_start[a]+dist_fr_start[b] , withpay + min([dist_fr_start[i] for i in path_a2b]))
    for i in range(1, n+1):
        dt,_ = dijkstra(graph,i)

        answer.append(dt[s] + dt[a] + dt[b])

    return min(answer)

# fares = [[4, 1, 10], [3, 5, 24], [5, 6, 2], [3, 1, 41], [5, 1, 24], [4, 6, 50], [2, 4, 66], [2, 3, 22], [1, 6, 25]]
fares=	[[2,6,6], [6,3,7], [4,6,7], [6,5,11], [2,5,12], [5,3,20], [2,4,8], [4,3,9]]
# fares = [[5, 7, 9], [4, 6, 4], [3, 6, 1], [3, 2, 3], [2, 1, 6]]

n,s,a,b = 6,4,6,2
# n,s,a,b = 7,3,4,1
solution(n,s,a,b,fares)

13

In [ ]:
from heapq import heappush, heappop

INF = int(1e9)
def dijkstra(graph, src):
    pq = [[0, src]]
    dist = [INF for _ in range(len(graph))]
    dist[src] = 0
    while pq:
        w, x = heappop(pq)
        if dist[x] < w:
            continue
        for y, cost in graph[x]:
            cost += w
            if cost < dist[y]:
                dist[y] = cost
                heappush(pq, [cost, y])
    return dist


def solution(n, s, a, b, fares):
    graph = [[] for _ in range(n + 1)]
    ans = INF
    for src, dst, cost in fares:
        graph[src].append((dst, cost))
        graph[dst].append((src, cost))
    for k in range(1, n + 1):
        dist_k = dijkstra(graph, k)
        ans = min(ans, dist_k[s] + dist_k[a] + dist_k[b])
    return ans

In [ ]:

def findFare(fares,fr,to):#노드 두개를 받으면, 그 사이 간선의 가중치를 구해주는 함수
    for fare in fares:
        if fr in fare:
            if to in fare:
                return fare[2]

def fareToGraph(fares):#fares가 들어오면 그걸 그래프(딕셔너리)로 만들어주는 함수
    dic={}
    for fare in fares:
        tmp=dic.get(fare[0])
        if tmp:
            tmp.extend([fare[1]])
        else:
            dic[fare[0]]=[fare[1]]
        tmp=dic.get(fare[1])
        if tmp:
            tmp.extend([fare[0]])
        else:
            dic[fare[1]]=[fare[0]]
    return dic

def Dijkstra(graph,fare,start,destiny):
    #시작과 동시에 각 정점까지의 거리를 딕셔너리에 생성 및 저장, INF로 초기화
    distance= {vertex : [float('inf'), start] for vertex in graph}
    
    #그래프 시작점은 0으로 초기화
    distance[start] = [0,start]
    #heap = heap.sorted(heap,key= lambda x : x[0])
    
    #모든 정점이 저장될 큐 생성
    queue=[]
    
    #queue.append([distance[start][0], start])
    queue.append([0, start])
    
    while queue:
        #우선순위큐(힙) 대신 매 이터레이션마다 소팅을함 ( pop() 을 하면 제일 작은게 튀어나옴 )
        queue.sort(reverse=True, key=lambda x : x[0])
        tmp=queue.pop()
        curDistance=tmp[0]
        curVertex=tmp[1]
        
        #업데이트 하지 않는경우 continue로 팝해서 꺼내버리고 while문 재실행
        if distance[curVertex][0] < curDistance : continue
        
        for adjacent in graph[curVertex]:
            weight = findFare(fare,curVertex,adjacent)
            newDistance = curDistance + weight
            
            
            # UPDATE
            if newDistance < distance[adjacent][0]:
                distance[adjacent] = [newDistance,curVertex]
                queue.append([newDistance,adjacent])
    
    #########################
    #백트래킹으로 경로찾기 (1단계 스택넣기)
    back=destiny
    tmppath=[destiny,]
    while distance[back][1] != start:
        back=distance[back][1]
        tmppath.append(back)
    tmppath.append(start)
    path=[]
    while tmppath:
        path.append(tmppath.pop())
    
    return path,distance[destiny][0]

def sharedRide(graph,fares,s,a,b):
    aTob=Dijkstra(graph,fares,a,b)
    tmp=[]
    for middle in aTob[0]:
        tmp.append(Dijkstra(graph,fares,s,middle))
    tmp.sort(key=lambda x : x[1])
    # print('([최저합승구간] , 최저합승가격) : ',tmp[0])
    return aTob[1] + tmp[0][1]

def solution(n,s,a,b,fares):
    graph=fareToGraph(fares)
    # print(graph)
    aRoute,apay=Dijkstra(graph,fares,s,a)
    bRoute,bpay=Dijkstra(graph,fares,s,b)
    shared = sharedRide(graph,fares,s,a,b)
    if apay+bpay > shared :
        return shared
    else : return apay+bpay